# 环境初始化

In [1]:
import aie

from aiearth.core.error import aie_error

save_img = True
# aie.Authenticate(token='ff0859af79711070b75b93848452399e')

# 如果在AIearthNotebook环境运行或本地环境变量中已经有了可以不用写
aiearth_token = "ff0859af79711070b75b93848452399e" 

# 鉴权初始化

In [2]:
try:
    import os
    token = aiearth_token or os.environ.get('aiearth_token') 

    if token is None:
        print('尝试获取token...')
        aie.Authenticate()
    else:
        print('token:',token)
        aie.Authenticate(token=token)
        
    aie.Initialize()
    
except AttributeError as e:
    print(e)
    assert False,'项目没有初始化或者aie-sdk可能没有被正确安装！'

# catching classes that do not inherit from BaseException is not allowed
# except aie_error as e:
#     print(e)
    
except Exception as e:
    print(e)    

token: ff0859af79711070b75b93848452399e
计算资源初始化中，请等待...
计算资源初始化完成.


# 确定roi

In [3]:

feature_collection_1 = aie.FeatureCollection('China_Province') \
                        .filter(aie.Filter.eq('province', '广西壮族自治区'))

feature_collection_2 = aie.FeatureCollection('China_Province') \
                        .filter(aie.Filter.eq('province', '广东省'))

union_fc = feature_collection_1.merge(feature_collection_2).union()
union_geometry = union_fc.geometry()

# feature的导出方式和image不同，要单独写
# aie.Export.feature.toAsset(union_fc, "union_fc").start()

# 地图初始化

In [4]:
map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

vis_params = {
    'color': '#00FF00'
}

map.addLayer(
    union_fc,
    vis_params,
    'region',
    bounds=union_fc.getBounds()
)
map

Map(center=[23.30590084, 110.87501098], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

# 辅助函数

In [5]:
from math import sqrt

img_all = {}

def get_max_and_min(img_target:aie.Image,region:aie.Geometry=union_geometry):
    reducer = aie.Reducer.max().combine(reducer2=aie.Reducer.min(), sharedInputs=True)
    result = img_target.reduceRegion(reducer, region)
    max,min = list(result.getInfo().values())
    return max,min

def fix_with_mean(img_target:aie.Image,region:aie.Geometry=union_geometry):
    reducer = aie.Reducer.mean()
    result = img_target.reduceRegion(reducer, union_geometry)

    mean_tag = list(result.getInfo().keys())[0]
    target_mean = result.getInfo()[mean_tag] 
    
    img_mean = aie.Image(target_mean).clip(union_geometry)
    img_target = img_target.unmask(img_mean)
    img_target = img_target.updateMask(img_target.mask())
    
    return img_target


def save(img_target:aie.Image, filename:str=None):
    if filename is None:
        filename = img_target.bandNames().getInfo()[1]
    
    # 获取分辨率
    reducer = aie.Reducer.min()
    result = img_target.pixelArea().reduceRegion(reducer, union_geometry)
    area = sqrt(result.getInfo()['area_min'])
    
    name = img_target.bandNames().getInfo()[0]
    print(name + ":" + str(area))

    # 导出到持久化空间
    aie.Export.image.toAsset(img_target, filename, area).start()
        
        
def add_img(img_target:aie.Image, map:aie.Map=map, vis_params:dict=None):
    name = img_target.bandNames().getInfo()[0]
        
    if not vis_params:
        vis_params = {
            'palette': [
                '0,0,255','0,255,255','0,255,0','255,255,0','255,0,0'
            ]
        }
    if not 'max' in vis_params.keys():
        max,min = get_max_and_min(img_target)
        vis_params['max'] = max
        vis_params['min'] = min
    if not 'bands' in vis_params.keys():
        vis_params['bands'] = name
    
    map.addLayer(
        img_target,
        vis_params,
        name,
        bounds=union_fc.getBounds()
    )
    
    if save_img:
        save(img_target, name)    
        
    img_all[name] = img_target

# 获取数据集

In [7]:
# 月度火烧迹地产品（MCD64A1 v006）
# https://engine-aiearth.aliyun.com/#/dataset/MODIS_MCD64A1_006

dataset_firepoint = aie.ImageCollection('MODIS_MCD64A1_006') \

# 用or运算提取所有火点
img_firepoint = dataset_firepoint.select(['BurnDate']).Or() \
                            .mask() \
                            .clip(union_geometry) \
                            .rename(["firepoint"])


vis_params = {
    'palette': ["0,0,0","255,0,0"],
    'min': 0,
    'max': 1,
}

add_img(img_firepoint, map, vis_params)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# DEM
# https://engine-aiearth.aliyun.com/#/dataset/Copernicus_DEM_30M
dataset_DEM = aie.ImageCollection('Copernicus_DEM_30M') \
            .filterBounds(union_fc) \
            .select(['elevation'])

img_DEM = dataset_DEM.mosaic().clip(union_geometry)
add_img(img_DEM, map, vis_params)

# 坡度和坡向
img_slope = aie.Terrain.slope(img_DEM).rename(["slope"])
add_img(img_slope, map, vis_params)
img_aspect = aie.Terrain.aspect(img_DEM).rename(["aspect"])
add_img(img_aspect, map, vis_params)
map

In [ ]:
# 中国逐年土地覆盖数据集（CLCD）
# https://engine-aiearth.aliyun.com/#/dataset/ESA_WORLD_COVER_V200

clcd_dataset = aie.ImageCollection('CHINA_CLCD_V01_WHU_IRSIP') \
                .filterBounds(union_geometry) \
                .filterDate('2021-01-01', '2021-12-31')


img_clcd = clcd_dataset.mosaic() \
                        .clip(union_geometry) \
                        .select(['Map']) \
                        .rename(["clcd"])

vis_params = {
    'min': 1,
    'max': 9,
    'palette': [
        '#fae39c','#446f33','#33a02c','#abd37b',
        '#1e69b4','#a6cee3','#cfbda3','#e24290',
        '#289be8'
    ]
}

add_img(img_clcd, map, vis_params)
map

In [ ]:
dataset_ndvi = aie.ImageCollection('MODIS_MOD13Q1_061') \
             .filterDate('2021-01-01', '2021-12-31') \
             .select(['NDVI']) 

img_ndvi = dataset_ndvi.mean() \
                        .clip(union_geometry) \
                        .rename(["NDVI"])


vis_params = {
    'palette': [ 
        '#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718',
        '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201',
        '#004C00', '#023B01', '#012E01', '#011D01', '#011301'
    ]
}

add_img(img_ndvi, map, vis_params)
map

In [ ]:
# ERA5-Land monthly averaged data
# https://engine-aiearth.aliyun.com/#/dataset/ERA5_LAND_MONTHLY
# 月度生物气候变量

dataset_Bioclimatic = aie.ImageCollection('ERA5_LAND_MONTHLY') \
                        .filterDate('2021-01-01', '2021-12-31') \
                        .filterBounds(union_fc)

# 平均2m温度             
dataset_avg_temp = dataset_Bioclimatic.select(['temperature_2m'])
img_avg_temp = dataset_avg_temp.mean().clip(union_geometry).rename(["avg_temp"])
# 平均地表温度
dataset_skin_avg_temp = dataset_Bioclimatic.select(['skin_temperature'])
img_skin_avg_temp = dataset_skin_avg_temp.mean().clip(union_geometry).rename(["skin_avg_temp"])
# 平均气压
dataset_surface_pressure = dataset_Bioclimatic.select(['surface_pressure'])
img_surface_pressure = dataset_surface_pressure.mean().clip(union_geometry).rename(["surface_pressure"])
# 年降水量
dataset_precipitation = dataset_Bioclimatic.select(['total_precipitation'])
img_precipitation = dataset_precipitation.sum().clip(union_geometry).rename(["precipitation"])
# 年蒸发量
dataset_evaporation = dataset_Bioclimatic.select(['total_evaporation'])
img_evaporation = dataset_evaporation.sum().clip(union_geometry).rename(["evaporation"])


add_img(img_avg_temp, map, vis_params)
add_img(img_skin_avg_temp, map, vis_params)
add_img(img_surface_pressure, map, vis_params)
add_img(img_precipitation, map, vis_params)
add_img(img_evaporation, map, vis_params)

In [ ]:
# WET
# 湿度指标
dataset = aie.ImageCollection('LANDSAT_LC09_C02_T1_L1') \
             .filterBounds(union_fc) \
             .filterDate('2021-01-01', '2022-12-31') \
             # .filter(aie.Filter.lte('eo:cloud_cover', 30.0))

# 去云算法
# https://engine-aiearth.aliyun.com/docs/page/case?d=6cf4e8
def removeLandsatCloud(image):
    # cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0))
    return image.updateMask(mask)

# def applyScaleFactors(image):
#     opticalBands = image.select('SR_B.').multiply(aie.Image(0.0000275)).add(aie.Image(-0.2))
#     thermalBands = image.select('ST_B.*').multiply(aie.Image(0.00341802)).add(aie.Image(149.0))
#     return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# images_no_cloud = dataset.map(removeLandsatCloud).map(applyScaleFactors)
images_no_cloud = dataset.map(removeLandsatCloud)

image_L9C2L2 = images_no_cloud.mosaic().clip(union_geometry)

# WET = 0.1511 B1 + 0.1973 B2 + 0.3283 B3 + 0.3407 B4 - 0.7171 B5 - 0.4559 B6
# http://www.yndxxb.ynu.edu.cn/yndxxbzrkxb/article/doi/10.7540/j.ynu.20190174?viewType=HTML
# https://zhuanlan.zhihu.com/p/479851365

# L9C2L1
img_wet_tmp = image_L9C2L2.select('B2').multiply(aie.Image(0.1511)) \
    .add(image_L9C2L2.select('B3').multiply(aie.Image(0.1973))) \
    .add(image_L9C2L2.select('B4').multiply(aie.Image(0.3283))) \
    .add(image_L9C2L2.select('B5').multiply(aie.Image(0.3407))) \
    .add(image_L9C2L2.select('B6').multiply(aie.Image(0.7171))) \
    .add(image_L9C2L2.select('B7').multiply(aie.Image(0.4559))) \
    .rename(["wet"])

img_wet = fix_with_mean(img_wet_tmp)

img_wet = img_wet_tmp.clip(union_geometry)

vis_params = {
    'palette': [ 
        '255,255,0', '0,0,255',
    ]
}

add_img(img_wet, map, vis_params)

In [ ]:
# WorldPop Estimated Residential Population
# https://engine-aiearth.aliyun.com/#/dataset/WORLDPOP_GP_100M_POP

dataset_pop = aie.ImageCollection('WORLDPOP_GP_100M_POP') \
                .filterDate('2021-01-01','2021-12-31') \
                .filterBounds(union_geometry) \
                .select(['population'])


img_pop = dataset_pop.mosaic().clip(union_geometry)

vis_params = {
      'palette': ['#24126c', '#1fff4f', '#d4ff50']
}

add_img(img_pop, map, vis_params)

In [ ]:
# OpenLandMap Soil Texture Class
# https://engine-aiearth.aliyun.com/#/dataset/OPENLANDMAP_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_V02

img_soil = aie.Image('OPENLANDMAP_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_V02')

img_soil = img_soil.select(['b0']).clip(union_geometry).rename(["soil"])

vis_params = {
    'bands': 'soil',
    'min': 1.0,
    'max': 12.0,
    'palette': [
        "#d5c36b","#b96947","#9d3706","#ae868f","#f86714","#46d143",
        "#368f20","#3e5a14","#ffd557","#fff72e","#ff5a9d","#ff005b",
  ]
}

add_img(img_soil, map, vis_params)

# 自适应分类模型

In [ ]:
# 采样

label = "firepoint"
img_firepoint = img_firepoint.rename([label])

img_all = img_firepoint.addBands(img_aspect) \
                        .addBands(img_slope) \
                        .addBands(img_DEM)  \
                        .addBands(img_pop)  \
                        .addBands(img_precipitation) \
                        .addBands(img_clcd) \
                        .addBands(img_soil) \
                        .addBands(img_surface_pressure) \
                        .addBands(img_avg_temp) \
                        .addBands(img_ndvi) \
                        .addBands(img_evaporation) \
                        .addBands(img_wet) \
                        .clip(union_geometry)
                        
                        
                        
# 确定取样范围
samples = img_all.sampleRegion(union_geometry, 300, True)

# 在训练样本中增加一列随机数, 选取80%的样本为训练样本, 选取20%的样本为验证样本
sample = samples.randomColumn()
training_sample = sample.filter(aie.Filter.lte('random', 0.70))
validation_sample = sample.filter(aie.Filter.gt('random', 0.70))

In [ ]:
# 创建自适应集成分类器，并进行训练
trained_classifier = aie.Classifier.adaBoost(10)
trained_classifier = trained_classifier.train(training_sample, 
                                                 label, 
                                                 img_all.bandNames().getInfo())

# 获取训练样本的混淆矩阵, 并对训练精度进行评估
train_accuracy = trained_classifier.confusionMatrix()
print('Training error matrix:', train_accuracy.getInfo())
print('Training overall accuracy:', train_accuracy.accuracy().getInfo())


# 使用验证集对分类器进行评估
validation = validation_sample.classify(trained_classifier)
validation_accuracy = validation.errorMatrix(label, 'classification')
print('Validation error matrix:', validation_accuracy.getInfo())
print('Validation accuracy:', validation_accuracy.accuracy().getInfo())

# 使用训练好的分类器对影像进行分类
img_classified = img_all.classify(trained_classifier)


In [ ]:
# 自适应集成分类图层
print(img_classified.getInfo())

map.addLayer(
    img_classified,
    {
        'min': 0,
        'max': 1,
        'palette': ["0,255,255","255,0,0"]
    },
    'aie_classification',
    bounds=union_fc.getBounds()
)

add_img(img_classified, map, vis_params)